# Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import keras
from keras.layers import Input, Flatten, Dense, LeakyReLU, Dropout
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras import backend as K
from keras.models import clone_model
from keras import Model
from keras.datasets import mnist
from keras.utils import np_utils

import tensorflow.compat.v1 as tf
from tensorflow.keras import regularizers
from tensorflow.keras import initializers

import numpy as np
import collections
import copy
import pandas as pd 
import matplotlib.pyplot as plt 
import scipy.optimize as op 
import seaborn as sns

import pickle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
prefix = '/content/gdrive/My Drive/'

Mounted at /content/gdrive


### Sample Utility

In [3]:
# utility_func_args = [x_train, y_train, x_val, y_val]
def sample_utility(n, size_min, size_max, utility_func, utility_func_args, random_state, ub_prob=0.2, verbose=False):

  x_train, y_train, x_val, y_val = utility_func_args

  X_feature_test = []
  y_feature_test = []

  x_train = np.array(x_train)
  y_train = np.array(y_train)

  N = len(y_train)

  np.random.seed(random_state)
  
  for i in range(n):
    if verbose:
      print('{} / {}'.format(i, n))

    n_select = np.random.choice(range(size_min, size_max))

    subset_index = []

    """
    if unbalance:
      n_per_class = int(N / 10)
      alpha = np.ones(10)
      alpha[np.random.choice(range(10))] = np.random.choice(range(1, 50))
    else:
      alpha = np.random.choice(range(1, 20), size=10, replace=True)
    """

    toss = np.random.uniform()

    # With probability ub_prob, sample a class-imbalanced subset
    if toss > 1-ub_prob:
      n_per_class = int(N / 10)
      alpha = np.ones(10)
      alpha[np.random.choice(range(10))] = np.random.choice(range(1, 50))
    else:
      alpha = np.random.choice(range(90, 100), size=10, replace=True)

    p = np.random.dirichlet(alpha=alpha)
    occur = np.random.choice(range(10), size=n_select, replace=True, p=p)
    counts = np.array([np.sum(occur==i) for i in range(10)])
    
    for i in range(10):
      ind_i = np.where(np.argmax(y_train, 1)==i)[0]
      if len(ind_i) > counts[i]:
        selected_ind_i = np.random.choice(ind_i, size=counts[i], replace=False)
      else:
        selected_ind_i = np.random.choice(ind_i, size=counts[i], replace=True)
      subset_index = subset_index + list(selected_ind_i)

    subset_index = np.array(subset_index)

    y_feature_test.append(utility_func(x_train[subset_index], y_train[subset_index], x_val, y_val))
    X_feature_test.append( subset_index )

  return X_feature_test, y_feature_test


def sample_utility_veryub(n, size_min, size_max, utility_func, utility_func_args, random_state, ub_prob=0.2, verbose=False):

  x_train, y_train, x_val, y_val = utility_func_args

  X_feature_test = []
  y_feature_test = []

  x_train = np.array(x_train)
  y_train = np.array(y_train)

  N = len(y_train)

  np.random.seed(random_state)
  
  for i in range(n):
    if verbose:
      print('{} / {}'.format(i, n))

    n_select = np.random.choice(range(size_min, size_max))

    if n_select > 0:
      subset_index = []

      toss = np.random.uniform()

      # With probability ub_prob, sample a class-imbalanced subset
      if toss > 1-ub_prob:
        alpha = np.random.choice(range(1, 100), size=10, replace=True)
      else:
        alpha = np.random.choice(range(90, 100), size=10, replace=True)

      p = np.random.dirichlet(alpha=alpha)
      occur = np.random.choice(range(10), size=n_select, replace=True, p=p)
      counts = np.array([np.sum(occur==i) for i in range(10)])
      
      for i in range(10):
        ind_i = np.where(np.argmax(y_train, 1)==i)[0]
        if len(ind_i) > counts[i]:
          selected_ind_i = np.random.choice(ind_i, size=counts[i], replace=False)
        else:
          selected_ind_i = np.random.choice(ind_i, size=counts[i], replace=True)
        subset_index = subset_index + list(selected_ind_i)

      subset_index = np.array(subset_index)

      y_feature_test.append(utility_func(x_train[subset_index], y_train[subset_index], x_val, y_val))
      X_feature_test.append( subset_index )
    else:
      y_feature_test.append(0.1)
      X_feature_test.append( np.array([]) )

  return X_feature_test, y_feature_test

### Deep Sets

In [4]:
class DeepSet(nn.Module):

    def __init__(self, in_features, set_features=128, hidden_ext=128, hidden_reg=128):
        super(DeepSet, self).__init__()
        self.in_features = in_features
        self.out_features = set_features
        self.feature_extractor = nn.Sequential(
            nn.Linear(in_features, hidden_ext, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_ext, hidden_ext, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_ext, set_features, bias=False)
        )

        self.regressor = nn.Sequential(
            nn.Linear(set_features, hidden_reg, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_reg, hidden_reg, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_reg, int(hidden_reg/2), bias=False),
            nn.ELU(inplace=True)
        )

        self.linear = nn.Linear(int(hidden_reg/2), 1)
        self.sigmoid = nn.Sigmoid()
        
        self.add_module('0', self.feature_extractor)
        self.add_module('1', self.regressor)
        
    def reset_parameters(self):
        for module in self.children():
            reset_op = getattr(module, "reset_parameters", None)
            if callable(reset_op):
                reset_op()
            
    def forward(self, input):
        x = input
        x = self.feature_extractor(x)
        x = x.sum(dim=1)
        x = self.regressor(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'Feature Exctractor=' + str(self.feature_extractor) \
            + '\n Set Feature' + str(self.regressor) + ')'


class DeepSet_cifar(nn.Module):

    def __init__(self, in_features, set_features=512):
        super(DeepSet_cifar, self).__init__()
        self.in_features = in_features
        self.out_features = set_features
        self.feature_extractor = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ELU(inplace=True),
            nn.Linear(512, 512),
            nn.ELU(inplace=True),
            nn.Linear(512, set_features)
        )

        self.regressor = nn.Sequential(
            nn.Linear(set_features, 512),
            nn.ELU(inplace=True),
            nn.Linear(512, 512),
            nn.ELU(inplace=True),
            nn.Linear(512, 512),
            nn.ELU(inplace=True),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
        self.add_module('0', self.feature_extractor)
        self.add_module('1', self.regressor)
        
        
    def reset_parameters(self):
        for module in self.children():
            reset_op = getattr(module, "reset_parameters", None)
            if callable(reset_op):
                reset_op()
            
    def forward(self, input):
        x = input
        x = self.feature_extractor(x)
        x = x.sum(dim=1)
        x = self.regressor(x)
        return x

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'Feature Exctractor=' + str(self.feature_extractor) \
            + '\n Set Feature' + str(self.regressor) + ')'

In [5]:
class DeepSet_dual(nn.Module):

    def __init__(self, in_features, set_features=128, hidden_ext=128, hidden_reg=128):
        super(DeepSet_dual, self).__init__()
        self.in_features = in_features
        self.out_features = set_features
        self.feature_extractor = nn.Sequential(
            nn.Linear(in_features, hidden_ext, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_ext, hidden_ext, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_ext, set_features, bias=False)
        )

        self.classifier = nn.Sequential(
            nn.Linear(set_features, hidden_reg, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_reg, hidden_reg, bias=False),
            nn.ELU(inplace=True),
            nn.Linear(hidden_reg, int(hidden_reg/2), bias=False),
            nn.ELU(inplace=True)
        )

        self.linear = nn.Linear(int(hidden_reg/2), 3) #NOTE: n_classes=3 as 0 <, 1 =, 2 >
        self.sigmoid = nn.Sigmoid()
        
        self.add_module('0', self.feature_extractor)
        self.add_module('1', self.classifier)
        
    def reset_parameters(self):
        for module in self.children():
            reset_op = getattr(module, "reset_parameters", None)
            if callable(reset_op):
                reset_op()
            
    def forward(self, input1, input2):
        x = torch.cat((input1, input2), dim=2)
        x = self.feature_extractor(x)
        x = x.sum(dim=1)
        x = self.classifier(x)
        x = self.linear(x)
        # x = self.sigmoid(x)
        return x

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'Feature Exctractor=' + str(self.feature_extractor) \
            + '\n Set Feature' + str(self.classifier) + ')'

### Deepset Utility Learning Model

In [6]:
class Utility_deepset_dual(object):

    def __init__(self, model=None):

        """
        if model is None:
          self.model = DeepSet(in_dims).cuda()
        else:
          self.model = model.cuda()
        """

        self.model = model

        self.model.linear.bias = torch.nn.Parameter(torch.tensor([-2.1972]))
        self.model.linear.bias.requires_grad = False
        #print(self.model.linear.bias)
        self.model.cuda()
        #print(self.model.linear.bias)
        
        self.l1 = nn.L1Loss()
        self.l2 = nn.MSELoss(reduction='sum')
        self.criterion = nn.CrossEntropyLoss()
        
    # train_data: x_train_few
    # train_set: (X_feature, y_feature)
    # valid_set: (X_feature_test, y_feature_test)
    def compare(self, y1, y2, tau=0.1):
      if y1 > y2+tau:
        return 2
      elif y2+tau > y1 and y1 >= y2-tau:
        return 1
      else:
        return 0

    def fit(self, train_data, train_set, valid_set, n_epoch, batch_size=32, lr=1e-3, n_pair=10000):

        self.optim = optim.Adam(self.model.parameters(), lr)

        #scheduler = StepLR(self.optim, step_size=10, gamma=0.1)
        scheduler = MultiStepLR(self.optim, milestones=[10,15], gamma=0.1)

        train_data = copy.deepcopy(train_data)
        N = train_data.shape[0]
        k = train_data.shape[1]

        X_feature, y_feature = train_set
        X_feature_test, y_feature_test = valid_set
        train_size = len(y_feature)

        for epoch in range(n_epoch):
          # sample paired inputs
          X_pair1, X_pair2, y_pair = [], [], []
          for s in range(n_pair):
            select = np.random.choice(train_size, size=2, replace=False)
            X_pair1.append(X_feature[select[0]])
            X_pair2.append(X_feature[select[1]])
            y_pair.append( self.compare(y_feature[select[0]], y_feature[select[1]], tau=0.1) )

          # train
          train_loss = 0
          for j in range(n_pair//batch_size):
            start_ind = j*batch_size
            batch_X1, batch_X2, batch_y = [], [], []
            for i in range(start_ind, min(start_ind+batch_size, n_pair)):
              b1, b2 = np.zeros((N, k)), np.zeros((N, k))
              if len(X_pair1[i]) > 0 and len(X_pair2[i] > 0):
                b1[:len(X_pair1[i])] = train_data[X_pair1[i]]
                b2[:len(X_pair2[i])] = train_data[X_pair2[i]]

              batch_X1.append( b1 )
              batch_X2.append( b2)
              batch_y.append( y_pair[i] )

            # print('tot{} batch {} length1 {} length2 {}'.format(n_pair//batch_size, j, len(batch_X1), len(batch_X2)))
            batch_X1 = np.stack(batch_X1)
            batch_X2 = np.stack(batch_X2)
            batch_X1, batch_X2, batch_y = torch.FloatTensor(batch_X1).cuda(), torch.FloatTensor(batch_X2).cuda(), torch.LongTensor(batch_y).cuda()

            self.optim.zero_grad()
            y_pred = self.model(batch_X1, batch_X2)
            # import pdb; pdb.set_trace()
            loss = self.criterion(y_pred, batch_y)
            loss_val = np.asscalar(loss.data.cpu().numpy())
            loss.backward()
            self.optim.step()
            train_loss += loss_val
          train_loss /= train_size
          test_loss = self.evaluate(train_data, valid_set, test_size=200)
          scheduler.step()
          print('Epoch %s Train Loss %s Test Loss %s' % (epoch, train_loss, test_loss))			
    
    def evaluate(self, train_data, valid_set, test_size=2000):

        N, k = train_data.shape
        X_feature_test, y_feature_test = valid_set

        X_pair1, X_pair2, y_pair = [], [], []
        for s in range(test_size):
          select = np.random.choice(len(y_feature_test), size=2, replace=False)
          X_pair1.append(X_feature_test[select[0]])
          X_pair2.append(X_feature_test[select[1]])
          y_pair.append( self.compare(y_feature_test[select[0]], y_feature_test[select[1]]) )


        test_loss = 0
        for i in range(test_size):
            b1, b2 = np.zeros((N, k)), np.zeros((N, k))
            if len(X_pair1[i]) > 0 and len(X_pair2[i]) > 0:
              b1[:len(X_pair1[i])] = train_data[X_pair1[i]]
              b2[:len(X_pair2[i])] = train_data[X_pair2[i]]

            batch_X1, batch_X2, batch_y = torch.FloatTensor(b1).cuda(), torch.FloatTensor(b2).cuda(), torch.LongTensor(y_pair[i:i+1]).cuda()
            batch_X1, batch_X2, batch_y = batch_X1.reshape((1, N, k)), batch_X2.reshape((1, N, k)), batch_y.reshape(-1)
            y_pred = self.model(batch_X1, batch_X2)

            loss = self.criterion(y_pred, batch_y)
            loss_val = np.asscalar(loss.data.cpu().numpy())
            test_loss += loss_val
        test_loss /= test_size
        return test_loss

In [7]:
import time
class findMostValuableSample_dual(object):
  def __init__(self, model, unlabeled_set, target_size, n_sample=5000):

      self.model = model
      self.unlabeled_set = unlabeled_set
      self.target_size = target_size
      self.n_sample = n_sample

  def compare(self, s1, s2):
    assert len(s1) == len(s2)
    # N = self.unlabeled_set.shape[0]
    # k = self.n_sample
    # b1, b2 = np.zeros((N, k)), np.zeros((N, k))
    # b1[:len(s1)] = self.unlabled_set[s1]
    # b2[:len(s2)] = self.unlabled_set[s2]
    input_dim = self.unlabeled_set.shape[1]
    x1 = torch.FloatTensor(self.unlabeled_set[s1].reshape((1, len(s1), input_dim))).cuda()
    x2 = torch.FloatTensor(self.unlabeled_set[s2].reshape((1, len(s2), input_dim))).cuda()
    y_pred = self.model(x1, x2)
    return torch.argmax(y_pred, dim=1).view(-1)

  def merge_sort(self, lst):
    if len(lst) <= 1:
      return lst
    mid = len(lst) // 2
    left = self.merge_sort(lst[:mid])
    right = self.merge_sort(lst[mid:])
    return self.merge(left, right)

  def merge(self, left, right):
    res = []
    i = 0
    j = 0
    while i < len(left) and j < len(right):
      if self.compare(left[i], right[j]) == 2:
        res.append(left[i])
      else:
        res.append(right[i])
    res += left[i:]
    res += right[j:]
    return res
  
  def get_rank(self):
    start = time.time()
    N = self.unlabeled_set.shape[0] #unlabel_size = 2000
    X_s = []
    for i in range(self.n_sample):
      select = np.random.choice(N, self.target_size, replace=False)
      X_s.append(select)
    
    rank = self.merge_sort(X_s)
    print("Time: {:.2f}".format(time.time()-start))
    return rank

In [8]:
class Utility_deepset(object):

    def __init__(self, model=None):

        """
        if model is None:
          self.model = DeepSet(in_dims).cuda()
        else:
          self.model = model.cuda()
        """

        self.model = model

        self.model.linear.bias = torch.nn.Parameter(torch.tensor([-2.1972]))
        self.model.linear.bias.requires_grad = False
        #print(self.model.linear.bias)
        self.model.cuda()
        #print(self.model.linear.bias)
        
        self.l1 = nn.L1Loss()
        self.l2 = nn.MSELoss(reduction='sum')
        
    # train_data: x_train_few
    # train_set: (X_feature, y_feature)
    # valid_set: (X_feature_test, y_feature_test)
    def fit(self, train_data, train_set, valid_set, n_epoch, batch_size=32, lr=1e-3):

        self.optim = optim.Adam(self.model.parameters(), lr)

        #scheduler = StepLR(self.optim, step_size=10, gamma=0.1)
        scheduler = MultiStepLR(self.optim, milestones=[10,15], gamma=0.1)

        train_data = copy.deepcopy(train_data)
        N = train_data.shape[0]
        k = train_data.shape[1]

        X_feature, y_feature = train_set
        X_feature_test, y_feature_test = valid_set
        train_size = len(y_feature)

        for epoch in range(n_epoch):

          # Shuffle training utility samples
          ind = np.arange(train_size, dtype=int)
          np.random.shuffle(ind)
          X_feature = [X_feature[i] for i in ind]
          y_feature = y_feature[ind]

          train_loss = 0
          start_ind = 0

          for j in range(train_size//batch_size):
            start_ind = j*batch_size
            batch_X, batch_y = [], []
            for i in range(start_ind, min(start_ind+batch_size, train_size)):

              b = np.zeros((N, k))
              if len(X_feature[i]) > 0:
                selected_train_data = train_data[X_feature[i]]
                b[:len(X_feature[i])] = selected_train_data

              batch_X.append( b )
              batch_y.append( [y_feature[i]] )

            batch_X = np.stack(batch_X)
            batch_X, batch_y = torch.FloatTensor(batch_X).cuda(), torch.FloatTensor(batch_y).cuda()

            self.optim.zero_grad()
            y_pred = self.model(batch_X)
            loss = self.l2(y_pred, batch_y)
            loss_val = np.asscalar(loss.data.cpu().numpy())
            loss.backward()
            self.optim.step()
            train_loss += loss_val
          train_loss /= train_size
          test_loss = self.evaluate(train_data, valid_set)
          scheduler.step()
          print('Epoch %s Train Loss %s Test Loss %s' % (epoch, train_loss, test_loss))
    
    def evaluate(self, train_data, valid_set):

        N, k = train_data.shape
        X_feature_test, y_feature_test = valid_set

        test_size = len(y_feature_test)
        test_loss = 0

        for i in range(test_size):

            b = np.zeros((N, k))
            if len(X_feature_test[i]) > 0:
              selected_train_data = train_data[X_feature_test[i]]
              b[:len(X_feature_test[i])] = selected_train_data

            batch_X, batch_y = torch.FloatTensor(b).cuda(), torch.FloatTensor(y_feature_test[i:i+1]).cuda()
            import pdb; pdb.set_trace()
            batch_X, batch_y = batch_X.reshape((1, N, k)), batch_y.reshape((1, 1))
            y_pred = self.model(batch_X)

            loss = self.l2(y_pred, batch_y)
            loss_val = np.asscalar(loss.data.cpu().numpy())
            test_loss += loss_val
        test_loss /= test_size
        return test_loss


def array_to_lst(X_feature):

  if type(X_feature) == list:
    return X_feature

  X_feature = list(X_feature)
  for i in range(len(X_feature)):
    X_feature[i] = X_feature[i].nonzero()[0]
  return X_feature


def findMostValuableSample_deepset_greedy(model, unlabeled_set, target_size):

  model = model.cuda()

  N, input_dim = unlabeled_set.shape
  k = target_size

  selected_subset = np.zeros(N)
  selected_rank = []
  selected_data = np.zeros((N, input_dim))

  for i in range(k):
    print(i)
    maxIndex, maxVal = -1, -1
    prevUtility = model(torch.FloatTensor(selected_data.reshape((1, N, input_dim))).cuda())
    searchRange = np.where(selected_subset == 0)[0]
    for j in searchRange:
      selected_subset[j] = 1
      selected_data[j] = unlabeled_set[j]
      utility = model(torch.FloatTensor(selected_data.reshape((1, N, input_dim))).cuda())
      selected_subset[j] = 0
      selected_data[j] = np.zeros(input_dim)
      if utility - prevUtility > maxVal:
        maxIndex = j
        maxVal = utility - prevUtility
    selected_subset[maxIndex] = 1
    selected_rank.append(maxIndex)
    selected_data[maxIndex] = unlabeled_set[maxIndex]

  return selected_subset, selected_rank, selected_data


def findMostValuableSample_deepset_stochasticgreedy(model, unlabeled_set, target_size, epsilon, seed, verbose=False, debug=False, label=None):

  model = model.cuda()

  N, input_dim = unlabeled_set.shape
  k = target_size

  selected_subset = np.zeros(N)
  selected_rank = []

  R = int((N/k)*np.log(1/epsilon))
  
  print('Sample Size R={}'.format(R))

  if debug:
    R = 10
    print('Sample Size R={}'.format(R))

  np.random.seed(seed)
  selected_data = np.zeros((N, input_dim))

  for i in range(k):
    if verbose: print(i)
    
    maxIndex, maxVal = -1, -1

    prevUtility = model(torch.FloatTensor(selected_data.reshape((1, N, input_dim))).cuda())
    searchRange = np.where(selected_subset == 0)[0]

    if debug:
      print('Step {}, prevUtility={}'.format(i, prevUtility))

    if R < len(searchRange):
      searchRange = np.random.choice(searchRange, size=R, replace=False)

    for j in searchRange:
      selected_subset[j] = 1
      selected_data[j] = unlabeled_set[j]
      utility = model(torch.FloatTensor(selected_data.reshape((1, N, input_dim))).cuda())
      selected_subset[j] = 0
      selected_data[j] = np.zeros(input_dim)
      gain = (utility - prevUtility).cpu().detach().numpy()[0][0]

      if gain > maxVal:
        maxIndex = j
        maxVal = gain

      if debug:
        print('  Gain from {} is {}, label={}'.format(j, gain, label[j]))
        print('  maxIndex={}, maxVal={}, labelMaxIndex={}'.format(maxIndex, maxVal, label[maxIndex]))
      
    selected_subset[maxIndex] = 1
    selected_rank.append(maxIndex)
    selected_data[maxIndex] = unlabeled_set[maxIndex]

  return selected_subset, selected_rank, selected_data

## Add Gaussian Noise

In [9]:
from google.colab.patches import cv2_imshow

def addGaussianNoise(x_train, scale=1):
  return np.clip(x_train+np.random.normal(scale=scale, size=x_train.shape), 0, 1)

## CNN Feature Extraction

In [10]:
class KerasLeNet:
    @staticmethod
    def build(numChannels, imgRows, imgCols, numClasses, activation="relu", weightsPath=None):

        model = Sequential()
        inputShape = (imgRows, imgCols, numChannels)
        if K.image_data_format() == "channels_first": 
          inputShape = (numChannels, imgRows, imgCols)

        if numChannels > 1:
          inputShape = (numChannels, imgRows, imgCols)

        model.add(Conv2D(20, 5, padding="same", input_shape=inputShape))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(40, 5, padding="same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(keras.layers.core.Flatten())
        model.add(Dense(500))
        model.add(Activation(activation))
        model.add(Dense(numClasses))
        model.add(Activation("softmax"))

        if weightsPath is not None:
            model.load_weights(weightsPath)

        return model


class KerasLeNetUSPS:
    @staticmethod
    def build(numChannels, imgRows, imgCols, numClasses, activation="relu", weightsPath=None):

        model = Sequential()
        inputShape = (imgRows, imgCols, numChannels)
        if K.image_data_format() == "channels_first": 
          inputShape = (numChannels, imgRows, imgCols)

        if numChannels > 1:
          inputShape = (numChannels, imgRows, imgCols)

        model.add(Conv2D(3, 5, padding="same", input_shape=inputShape))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(6, 5, padding="same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(keras.layers.core.Flatten())
        model.add(Dense(500))
        model.add(Activation(activation))
        model.add(Dense(numClasses))
        model.add(Activation("softmax"))

        if weightsPath is not None:
            model.load_weights(weightsPath)

        return model


def extractFeatures(layeredOutput, data):

    featureDataList = list()

    for idx in range(len(data)):
        x = data[idx:idx+1]
        featureVector = layeredOutput([x])[0][0]
        dataList = featureVector.tolist()
        featureDataList.append(dataList)

    return np.array(featureDataList)

In [ ]:
x_train = usps_x_train.reshape((len(usps_x_train), 16, 16, 1))
x_test = usps_x_test.reshape((len(usps_x_test), 16, 16, 1))

y_train = np_utils.to_categorical(usps_y_train, 10)
y_test = np_utils.to_categorical(usps_y_test, 10)

opt = tf.train.AdamOptimizer()
model = KerasLeNetUSPS.build(numChannels=1, imgRows=16, imgCols=16, numClasses=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
(loss, accuracy) = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))
# model.save_weights(save_dir+"/usps_featureExtractor.h5")

Epoch 1/20
57/57 [==============================] - 1s 8ms/step - loss: 1.7609 - accuracy: 0.4323 - val_loss: 0.5065 - val_accuracy: 0.8480
Epoch 2/20
57/57 [==============================] - 0s 4ms/step - loss: 0.3230 - accuracy: 0.9088 - val_loss: 0.3581 - val_accuracy: 0.9003
Epoch 3/20
57/57 [==============================] - 0s 4ms/step - loss: 0.2085 - accuracy: 0.9414 - val_loss: 0.3228 - val_accuracy: 0.9143
Epoch 4/20
57/57 [==============================] - 0s 5ms/step - loss: 0.1547 - accuracy: 0.9596 - val_loss: 0.3095 - val_accuracy: 0.9203
Epoch 5/20
57/57 [==============================] - 0s 5ms/step - loss: 0.1398 - accuracy: 0.9597 - val_loss: 0.2858 - val_accuracy: 0.9233
Epoch 6/20
57/57 [==============================] - 0s 5ms/step - loss: 0.1218 - accuracy: 0.9658 - val_loss: 0.2488 - val_accuracy: 0.9387
Epoch 7/20
57/57 [==============================] - 0s 4ms/step - loss: 0.1006 - accuracy: 0.9706 - val_loss: 0.2461 - val_accuracy: 0.9377
Epoch 8/20
57/57 [==

In [ ]:
model_dir = 'drive/MyDrive/active-learning-transfer/models'
model.save_weights(model_dir+"/usps_featureExtractor.h5")

In [11]:
model_dir = prefix + '/active-learning-transfer/models/usps_optimal'

opt = tf.train.AdamOptimizer()
extractor = KerasLeNetUSPS.build(numChannels=1, imgRows=16, imgCols=16, numClasses=10)
extractor.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
load_status = extractor.load_weights(model_dir+'/usps_featureExtractor.h5')
lastLayerOp = K.function([extractor.layers[0].input], [extractor.layers[6].output])

OSError: ignored

In [ ]:
x_train_few_cnnFeature = extractFeatures(lastLayerOp, usps_x_train_few.reshape((300, 16, 16, 1)))

# USPS

In [ ]:
import numpy as np 
import pandas as pd 
import os

import h5py 
from functools import reduce

def hdf5(path, data_key = "data", target_key = "target", flatten = True):
    """
        loads data from hdf5: 
        - hdf5 should have 'train' and 'test' groups 
        - each group should have 'data' and 'target' dataset or spcify the key
        - flatten means to flatten images N * (C * H * W) as N * D array
    """
    with h5py.File(path, 'r') as hf:
        train = hf.get('train')
        X_tr = train.get(data_key)[:]
        y_tr = train.get(target_key)[:]
        test = hf.get('test')
        X_te = test.get(data_key)[:]
        y_te = test.get(target_key)[:]
        if flatten:
            X_tr = X_tr.reshape(X_tr.shape[0], reduce(lambda a, b: a * b, X_tr.shape[1:]))
            X_te = X_te.reshape(X_te.shape[0], reduce(lambda a, b: a * b, X_te.shape[1:]))
    return X_tr, y_tr, X_te, y_te

In [ ]:
usps_dir = prefix + '/active-learning-transfer/usps/usps.h5'

usps_x_train, usps_y_train, usps_x_test, usps_y_test = hdf5(usps_dir)

In [ ]:
unlabel_size = 2000
target_size = 1000

np.random.seed(816)
usps_unlabel_idx = np.random.choice(range(len(usps_x_train)), size=unlabel_size, replace=False)
usps_x_unlabel, usps_y_unlabel = usps_x_train[usps_unlabel_idx], usps_y_train[usps_unlabel_idx]
usps_x_unlabel[:1700] = addGaussianNoise(usps_x_unlabel[:1700], scale=10)

x_unlabel_cnnFeature = extractFeatures(lastLayerOp, usps_x_unlabel.reshape((len(usps_x_unlabel), 16, 16, 1)))

In [ ]:
x_unlabel_cnnFeature.shape[1]

96

### sample utility

In [ ]:
np.random.seed(999)
X_feature, y_feature = sample_utility(n=5000, size_min=1, size_max=150, x_train=usps_x_train, y_train=usps_y_train, utility_func=uci_data_to_f1)

### load utiltiy

In [ ]:
sample_dir = prefix + '/active-learning-transfer/samples'

(usps_x_train_few, usps_y_train_few), usps_X_feature, usps_y_feature = pickle.load( open(sample_dir+'/USPS_svm_N300_lo5_hi300_DataSeed803_Nsample5000_Seed1.sample', 'rb') )

print(usps_x_train_few.shape)
print(len(usps_X_feature))

x_train_few_cnnFeature = extractFeatures(lastLayerOp, usps_x_train_few.reshape((300, 16, 16, 1)))

usps_y_feature = np.array(usps_y_feature)
usps_X_feature_test, usps_y_feature_test = usps_X_feature[4000:], usps_y_feature[4000:]
usps_X_feature, usps_y_feature = usps_X_feature[:4000], usps_y_feature[:4000]



input_dim = x_train_few_cnnFeature.shape[1]
print('input_dim:', input_dim)
set_features=128
hidden_ext=128
hidden_reg=128
n_epoch=50
batch_size=32
lr = 1e-4

model_usps = Utility_deepset_dual(model=DeepSet_dual(input_dim*2, set_features, hidden_ext, hidden_reg))

for e in range(n_epoch):
  model_usps.fit(x_train_few_cnnFeature, (usps_X_feature, usps_y_feature), (usps_X_feature_test, usps_y_feature_test), 1, batch_size, lr,  n_pair=20000)
  # _, pair_selected_rank, _ = findMostValuableSample_deepset_stochasticgreedy(model_usps.model, x_unlabel_cnnFeature, target_size, epsilon=1e-4, seed=101)
  rank_func = findMostValuableSample_dual(model_usps.model, x_unlabel_cnnFeature, target_size, n_sample=100)
  pair_selected_rank = rank_func.get_rank()
  print(sum(np.array(pair_selected_rank[0]) < 1700))
  print(sum(np.array(pair_selected_rank[1]) < 1700))
  print(sum(np.array(pair_selected_rank[2]) < 1700))
  print('------------')

(300, 256)
5000
input_dim: 96


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:117: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


Epoch 0 Train Loss 0.018082459373399615 Test Loss 0.19594939464368788


In [ ]:
sample_dir = prefix + '/active-learning-transfer/samples'

(usps_x_train_few, usps_y_train_few), usps_X_feature, usps_y_feature = pickle.load( open(sample_dir+'/USPS_svm_N300_lo5_hi300_DataSeed803_Nsample5000_Seed1.sample', 'rb') )

print(usps_x_train_few.shape)
print(len(usps_X_feature))

x_train_few_cnnFeature = extractFeatures(lastLayerOp, usps_x_train_few.reshape((300, 16, 16, 1)))

usps_y_feature = np.array(usps_y_feature)
usps_X_feature_test, usps_y_feature_test = usps_X_feature[4000:], usps_y_feature[4000:]
usps_X_feature, usps_y_feature = usps_X_feature[:4000], usps_y_feature[:4000]



input_dim = x_train_few_cnnFeature.shape[1]
set_features=128
hidden_ext=128
hidden_reg=128
n_epoch=50
batch_size=32
lr = 1e-4

model_optimal_usps = Utility_deepset(model=DeepSet(input_dim, set_features, hidden_ext, hidden_reg))
for e in range(n_epoch):
  model_optimal_usps.fit(x_train_few_cnnFeature, (usps_X_feature, usps_y_feature), (usps_X_feature_test, usps_y_feature_test), 1, batch_size, lr)
  _, optimal_dulo_selected_rank, _ = findMostValuableSample_deepset_stochasticgreedy(model_optimal_usps.model, x_unlabel_cnnFeature, target_size, epsilon=1e-4, seed=101)
  print(sum(np.array(optimal_dulo_selected_rank) < 1700))

(300, 256)
5000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


> <ipython-input-59-f11ed7ab18bb>(97)evaluate()
-> batch_X, batch_y = batch_X.reshape((1, N, k)), batch_y.reshape((1, 1))
(Pdb) print(batch_X.shape)
torch.Size([300, 96])
(Pdb) print(train_data.shape)
(300, 96)
(Pdb) quit


BdbQuit: ignored

In [ ]:
torch.save(model_optimal_usps.model.state_dict(), model_dir+'/usps_optimal.state_dict')

### Run Optimal

In [ ]:
prefix = '/content/gdrive/My Drive/'
model_dir = prefix + 'active-learning-transfer/models/usps_optimal'

opt = tf.train.AdamOptimizer()
extractor = KerasLeNetUSPS.build(numChannels=1, imgRows=16, imgCols=16, numClasses=10)
extractor.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
load_status = extractor.load_weights(model_dir+'/usps_featureExtractor.h5')
lastLayerOp = K.function([extractor.layers[0].input], [extractor.layers[6].output])

usps_dir = prefix + 'active-learning-transfer/usps/usps.h5'

usps_x_train, usps_y_train, usps_x_test, usps_y_test = hdf5(usps_dir)

unlabel_size = 2000
target_size = 1000

np.random.seed(816)
usps_unlabel_idx = np.random.choice(range(len(usps_x_train)), size=unlabel_size, replace=False)
usps_x_unlabel, usps_y_unlabel = usps_x_train[usps_unlabel_idx], usps_y_train[usps_unlabel_idx]
usps_x_unlabel[:1700] = addGaussianNoise(usps_x_unlabel[:1700], scale=10)

x_unlabel_cnnFeature = extractFeatures(lastLayerOp, usps_x_unlabel.reshape((len(usps_x_unlabel), 16, 16, 1)))

# input_dim = x_unlabel_cnnFeature.shape[1]
input_dim = 500
set_features=128
hidden_ext=128
hidden_reg=128
n_epoch=50
batch_size=32
lr = 1e-4

model_optimal_usps = Utility_deepset(model=DeepSet(input_dim, set_features, hidden_ext, hidden_reg))

model_optimal_usps.model.load_state_dict(torch.load(model_dir+'/usps_optimal_new.state_dict'))

_, optimal_dulo_selected_rank, _ = findMostValuableSample_deepset_stochasticgreedy(model_optimal_usps.model, x_unlabel_cnnFeature, target_size, epsilon=1e-4, seed=101)

print(sum(np.array(optimal_dulo_selected_rank) < 1700))

Sample Size R=18


RuntimeError: ignored

In [ ]:
print(type(optimal_dulo_selected_rank))
np.savetxt(model_dir+"/optimal_dulo_selected_rank_seed816_1000_new.csv", optimal_dulo_selected_rank, delimiter =",", fmt='%d')

<class 'list'>


In [ ]:
print(optimal_dulo_selected_rank)

[1333, 1712, 1948, 1704, 1295, 1878, 1820, 1886, 1900, 225, 1761, 1717, 1858, 328, 1293, 803, 691, 1777, 1921, 1997, 1548, 1961, 135, 1819, 1883, 1809, 1978, 1703, 1731, 1938, 1936, 1556, 693, 642, 1784, 812, 1985, 1729, 1987, 1930, 1905, 1805, 1898, 542, 1262, 1786, 246, 1723, 1984, 1783, 1956, 161, 1846, 1932, 1916, 1764, 1744, 1554, 1973, 1903, 1633, 1859, 1532, 1839, 1267, 1754, 1890, 1877, 448, 1829, 1899, 1862, 1995, 1982, 1871, 103, 575, 873, 1812, 1254, 1604, 1887, 1792, 1988, 1707, 1832, 1516, 1714, 1747, 1813, 1843, 1621, 1702, 1957, 1048, 239, 1725, 1855, 1866, 763, 1966, 1101, 451, 1914, 1826, 368, 618, 1710, 1852, 1773, 1976, 256, 1571, 478, 1849, 307, 1884, 1928, 1776, 1864, 1983, 734, 126, 1746, 450, 1891, 1685, 1771, 1741, 1711, 990, 679, 780, 1210, 1050, 1221, 1944, 1721, 262, 1906, 1857, 489, 1872, 855, 1724, 1889, 224, 1845, 1152, 1263, 1191, 484, 1759, 1986, 1909, 1781, 680, 1865, 1825, 927, 245, 1880, 1919, 1824, 1807, 406, 1403, 1748, 1827, 919, 1965, 1933, 259, 1

In [ ]:
print(model_dir)

/content/gdrive/My Drive/active-learning-transfer/models/usps_optimal


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_dir = prefix + '/active-learning-transfer/models/cycada/ds/iter_1_28_noise'
adda_net_file = model_dir + '/adda_LeNet_net_mnist_usps.pth'
net = get_model('AddaNet', num_cls=10, weights_init=adda_net_file, 
                model='LeNet')
net.eval()

def featureExtract_cycda(x, ext):
  assert list(x.shape[1:]) == [1, 28, 28]
  score, out = ext(x.cuda(), with_ft=True)
  return out.view(out.size(0), -1)

prefix = '/content/gdrive/My Drive/'
usps_dir = prefix + 'active-learning-transfer/usps/usps.h5'
usps_x_train, usps_y_train, usps_x_test, usps_y_test = hdf5(usps_dir)

unlabel_size = 2000
target_size = 1000

np.random.seed(816)
usps_unlabel_idx = np.random.choice(range(len(usps_x_train)), size=unlabel_size, replace=False)
usps_x_unlabel, usps_y_unlabel = usps_x_train[usps_unlabel_idx], usps_y_train[usps_unlabel_idx]
usps_x_unlabel[:1700] = addGaussianNoise(usps_x_unlabel[:1700], scale=10)

usps_x_unlabel_re, usps_y_unlabel_re, usps_x_test_re, usps_y_test_re = usps_encoderProcess(usps_x_unlabel, usps_y_unlabel, usps_x_test, usps_y_test)
x_unlabel_cnnFeature = featureExtract_cycda(usps_x_unlabel_re, net.tgt_net)
x_unlabel_cnnFeature = x_unlabel_cnnFeature.cpu().detach().numpy()

input_dim = x_unlabel_cnnFeature.shape[1]
set_features=128
hidden_ext=128
hidden_reg=128
n_epoch=50
batch_size=32
lr = 1e-4

model_optimal_usps = Utility_deepset(model=DeepSet(input_dim, set_features, hidden_ext, hidden_reg))

model_optimal_usps.model.load_state_dict(torch.load(model_dir+'/usps_optimal.state_dict'))

_, optimal_dulo_selected_rank, _ = findMostValuableSample_deepset_stochasticgreedy(model_optimal_usps.model, x_unlabel_cnnFeature, target_size, epsilon=1e-4, seed=101)

print(sum(np.array(optimal_dulo_selected_rank) < 1700))